<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from scTrace.pipeline import *

E:\Software\Anaconda\installation\envs\scTrace_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
runLabel = "Erlotinib"
savePath = "D:/Lineage-data/NBT21-TraceSeq/Results/"
dataPath = "D:/Lineage-data/NBT21-TraceSeq/ro_d4_erlotinib_1/"

In [3]:
adata = sc.read_10x_mtx(
    dataPath + 'filtered_feature_bc_matrix/',  # the directory with the `.mtx` file
    var_names = 'gene_symbols',    # use gene symbols for the variable names (variables-axis index)
    make_unique = True, cache=False)

In [4]:
cellAnno = pd.read_csv(dataPath + 'cellAnno.txt', sep = '\t')

C:\Users\ZeyuChen\AppData\Local\Temp\ipykernel_18436\104678391.py:1: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  cellAnno = pd.read_csv(dataPath + 'cellAnno.txt', sep = '\t')


In [5]:
adata.obs = cellAnno

In [6]:
adata

AnnData object with n_obs × n_vars = 27620 × 45066
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'cell', 'sample', 'barcode', 'reads_total', 'mean_quality', 'reads_mapped', 'reads_analyzed', 'reads_top_gene', 'umis_total', 'umis_top_gene', 'umis_mitochondrial', 'genes_detected', 'fraction_reads_mapped', 'fraction_reads_analyzed', 'fraction_reads_top_gene', 'fraction_umis_mitochondrial', 'fraction_umis_top_gene', 'min_umis', 'detected', 'max_mt', 'passed', 'sizefactor', 'S.Score', 'G2M.Score', 'Phase', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.6', 'RNA_snn_res.0.7', 'RNA_snn_res.0.8', 'RNA_snn_res.0.9', 'RNA_snn_res.1', 'seurat_clusters', 'run', 'G1S1', 'S2', 'G2M3', 'M4', 'MG15', 'G06', 'cc.expanded', 'gfpbc', 'GFP_class', 'PC_1', 'PC_2', 'UMAP_1', 'UMAP_2', 'TSNE_1', 'TSNE_2'
    var: 'gene_ids', 'feature_types'

In [9]:
adata.obs['run'].unique()

array(['ro_d0', 'ro_d4_erlotinib_1'], dtype=object)

In [8]:
adata

AnnData object with n_obs × n_vars = 27620 × 45066
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'cell', 'sample', 'barcode', 'reads_total', 'mean_quality', 'reads_mapped', 'reads_analyzed', 'reads_top_gene', 'umis_total', 'umis_top_gene', 'umis_mitochondrial', 'genes_detected', 'fraction_reads_mapped', 'fraction_reads_analyzed', 'fraction_reads_top_gene', 'fraction_umis_mitochondrial', 'fraction_umis_top_gene', 'min_umis', 'detected', 'max_mt', 'passed', 'sizefactor', 'S.Score', 'G2M.Score', 'Phase', 'RNA_snn_res.0.1', 'RNA_snn_res.0.2', 'RNA_snn_res.0.3', 'RNA_snn_res.0.4', 'RNA_snn_res.0.5', 'RNA_snn_res.0.6', 'RNA_snn_res.0.7', 'RNA_snn_res.0.8', 'RNA_snn_res.0.9', 'RNA_snn_res.1', 'seurat_clusters', 'run', 'G1S1', 'S2', 'G2M3', 'M4', 'MG15', 'G06', 'cc.expanded', 'gfpbc', 'GFP_class', 'PC_1', 'PC_2', 'UMAP_1', 'UMAP_2', 'TSNE_1', 'TSNE_2'
    var: 'gene_ids', 'feature_types'

In [28]:
def getScObjects(scobj, item, values, n_neighbors = 10, n_pcs = 40):
    data_split = [scobj[scobj.obs[item] == v,:] for v in values]
    data_split = [preProcess(d) for d in data_split]
    comm_hvgs = set([g for d in data_split for g in d.var.index[d.var.highly_variable]])
    
    for di in range(len(data_split)):
        data_split[di].var.highly_variable = [gene in comm_hvgs for gene in data_split[di].var.index]
        data_split[di] = runScanpy(data_split[di], n_neighbors, n_pcs, regress=False)
    
    return(data_split)

In [38]:
adata.obs['run'].unique()

['ro_d0', 'ro_d4_erlotinib_1']
Categories (2, object): ['ro_d0', 'ro_d4_erlotinib_1']

In [36]:
data0, data4 = getScObjects(adata, item = 'run', values = ['ro_d0', 'ro_d4_erlotinib_1'], n_neighbors = 10, n_pcs = 20)

/home/glab/wguo/softwares/anaconda3/envs/gwb-r40/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:170: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [37]:
data0.write(savePath + 'D0_scobj.h5ad')
data4.write(savePath + 'D4_erlo_scobj.h5ad')

In [ ]:
data0.obs['gfpbc'][data0.obs['gfpbc'] == "No_BC"] = np.nan
data0.obs['gfpbc'][data0.obs['gfpbc'] == "No_BC"] = np.nan

In [ ]:
data0.write(savePath + 'D0_scobj_update.h5ad')
data0.write(savePath + 'D4_scobj_update.h5ad')